<img src="https://github.com/insaid2018/Term-1/blob/master/Images/INSAID_Full%20Logo.png?raw=true" width="240" height="360" />

<center><h2>Capstone Project : HR Analytics on Employee Attrition & Retain employee</h2></center><br>
<center><h3>Identify employees, HR can be more proactive in reaching out to them before it's too late.</h3></center>

* <B>Created by : Sudeep Raj</B>

## Table of contents
1. Objective
2. Data Fields
3. Data Loading and Visualization
4. Conclusion

### 1. Objective

Your client for this project is the HR Department at a software company.

* They want to try a new initiative to retain employees.
* The idea is to use data to predict whether an employee is likely to leave.
* Once these employees are identified, HR can be more proactive in reaching out to them before it's too late.
* They only want to deal with the data that is related to permanent employees.

Current Practice
* Once an employee leaves, he or she is taken an interview with the name “exit interview” and shares reasons for leaving. The HR Department then tries and learns insights from the interview and makes changes accordingly.

This suffers from the following problems:
* This approach is that it's too haphazard. The quality of insight gained from an interview depends heavily on the skill of the interviewer.
* The second problem is these insights can't be aggregated and interlaced across all employees who have left.
* The third is that it is too late by the time the proposed policy changes take effect.

The HR department has hired you as data science consultants. They want to supplement their exit interviews with a more proactive approach.

Your Role
* You are given datasets of past employees and their status (still employed or already left).
* Your task is to build a classification model using the datasets.
* Because there was no machine learning model for this problem in the company, you don’t have quantifiable win condition. You need to build the best possible model.

### 2. Data Fields

The Business Intelligence Analysts of the Company provided you three datasets that contain information about past employees and their status (still employed or already left).

1. department_data

    This dataset contains information about each department. The schema of the dataset is as follows:
    * dept_id – Unique Department Code
    * dept_name – Name of the Department
    * dept_head – Name of the Head of the Department

2. employee_details_data

    This dataset consists of Employee ID, their Age, Gender and Marital Status. The schema of this dataset is as follows:
    * employee_id – Unique ID Number for each employee
    * age – Age of the employee
    * gender – Gender of the employee
    * marital_status – Marital Status of the employee

3. employee_data

    This dataset consists of each employee’s Administrative Information, Workload Information, Mutual Evaluation Information and Status.

    __Target variable__

    * status – Current employment status (Employed / Left)

    __Administrative information__

    * department – Department to which the employees belong(ed) to
    * salary – Salary level with respect to rest of their department
    * tenure – Number of years at the company
    * recently_promoted – Was the employee promoted in the last 3 years?
    * employee_id – Unique ID Number for each employee

    __Workload information__

    * n_projects – Number of projects employee has worked on
    * avg_monthly_hrs – Average number of hours worked per month

    __Mutual evaluation information__

    * satisfaction – Score for employee’s satisfaction with the company (higher is better)
    * last_evaluation – Score for most recent evaluation of employee (higher is better)
    * filed_complaint – Has the employee filed a formal complaint in the last 3 years?


### 3. Data Loading and Visualization

In [2]:
# Database 
import mysql.connector
from mysql.connector import Error

# ML
import pandas as pd 
import numpy as np
import matplotlib.pylab as plt 
%matplotlib inline   
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

In [3]:
tables = ['department_data', 'employee_details_data', 'employee_data']
try:
    mySQLconnection = mysql.connector.connect(host='projects.insaid.co', database='Capstone2', user='student', password='student')
    
    if mySQLconnection.is_connected():
        db_Info = mySQLconnection.get_server_info()
        print("Connected to MySQL database... MySQL Server version on ",db_Info)
        cursor = mySQLconnection.cursor()
        cursor.execute("select database();")
        database = cursor.fetchone()
        print ("Your connected to - ", database)
            
        sql_select_Query = "select * from department_data"
        SQL_Query = pd.read_sql_query(sql_select_Query, mySQLconnection)
        df_departmentData = pd.DataFrame(SQL_Query)
        print(df_departmentData.head(3))
        
        sql_select_Query = "select * from employee_details_data"
        SQL_Query = pd.read_sql_query(sql_select_Query, mySQLconnection)
        df_employeeDetailsData = pd.DataFrame(SQL_Query)
        print(df_employeeDetailsData.head(3))
        
        sql_select_Query = "select * from employee_data"
        SQL_Query = pd.read_sql_query(sql_select_Query, mySQLconnection)
        df_employeeData = pd.DataFrame(SQL_Query)
        print(df_employeeData.head(3))
        
except Error as e :
    print ("Error while connecting to MySQL", e)
finally:
    #closing database connection.
    if(mySQLconnection .is_connected()):
        mySQLconnection.close()
        print("MySQL connection is closed")

Connected to MySQL database... MySQL Server version on  5.7.25-0ubuntu0.18.04.2
Your connected to -  ('Capstone2',)
  dept_id dept_name        dept_head
0  D00-IT        IT       Henry Adey
1  D00-SS     Sales  Edward J Bayley
2  D00-TP      Temp  Micheal Zachrey
   employee_id  age  gender marital_status
0       113558   43    Male        Married
1       112256   24  Female      Unmarried
2       112586   22  Female      Unmarried
   avg_monthly_hrs department  filed_complaint  last_evaluation  n_projects  \
0            246.0       None              NaN         0.866838           6   
1            134.0       None              NaN         0.555718           2   
2            156.0     D00-SS              1.0         0.474082           2   

   recently_promoted  salary  satisfaction status  tenure  employee_id  
0                NaN  medium      0.134415   Left     4.0       124467  
1                NaN     low      0.511041   Left     3.0       112210  
2                NaN  medium

In [4]:
df_departmentData.shape

(11, 3)

In [5]:
df_departmentData.head(12)

,dept_id,dept_name,dept_head
0,D00-IT,IT,Henry Adey
1,D00-SS,Sales,Edward J Bayley
2,D00-TP,Temp,Micheal Zachrey
3,D00-ENG,Engineering,Sushant Raghunathan K
4,D00-SP,Support,Amelia Westray
5,D00-FN,Finance,Aanchal J
6,D00-PR,Procurement,Louie Viles
7,D00-AD,Admin,Evelyn Tolson
8,D00-MN,Management,Ellie Trafton
9,D00-MT,Marketing,Reuben Swann


In [6]:
df_employeeDetailsData.shape

(14245, 4)

In [7]:
df_employeeDetailsData.isnull().values.any()

False

No Null records present in __df_employeeDetailsData.__

In [8]:
df_employeeDetailsData.head()

,employee_id,age,gender,marital_status
0,113558,43,Male,Married
1,112256,24,Female,Unmarried
2,112586,22,Female,Unmarried
3,108071,36,Male,Married
4,116915,38,Male,Married


In [9]:
df_employeeData.shape

(14150, 11)

In [10]:
df_employeeData.head()

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id
0,246.0,None,NaN,0.866838,6,NaN,medium,0.134415,Left,4.0,124467
1,134.0,None,NaN,0.555718,2,NaN,low,0.511041,Left,3.0,112210
2,156.0,D00-SS,1.0,0.474082,2,NaN,medium,0.405101,Left,3.0,126150
3,256.0,D00-SP,NaN,0.961360,6,NaN,low,0.152974,Left,4.0,125346
4,146.0,D00-SS,NaN,0.507349,2,NaN,medium,0.434845,Left,3.0,113707


In [11]:
df_employeeData['department'].unique()

array([None, 'D00-SS', 'D00-SP', 'D00-MT', 'D00-PD', 'D00-IT', 'D00-AD',
       'D00-MN', 'D00-ENG', 'D00-PR', 'D00-TP', 'D00-FN', '-IT'],
      dtype=object)

We have to clean our data, and remove Null values.

#### Merged the data with respect to Employee id

In [12]:
df_emp = pd.merge(df_employeeData, df_employeeDetailsData[['employee_id', 'age', 'gender', 'marital_status']], on='employee_id', how='outer')
df_emp.head()

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,age,gender,marital_status
0,246.0,None,NaN,0.866838,6.0,NaN,medium,0.134415,Left,4.0,124467,42.0,Female,Married
1,134.0,None,NaN,0.555718,2.0,NaN,low,0.511041,Left,3.0,112210,23.0,Female,Unmarried
2,156.0,D00-SS,1.0,0.474082,2.0,NaN,medium,0.405101,Left,3.0,126150,24.0,Female,Unmarried
3,256.0,D00-SP,NaN,0.961360,6.0,NaN,low,0.152974,Left,4.0,125346,51.0,Female,Married
4,146.0,D00-SS,NaN,0.507349,2.0,NaN,medium,0.434845,Left,3.0,113707,23.0,Female,Unmarried


In [14]:
df_emp.shape

(14279, 14)

In [15]:
df_emp.describe(include='all')

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,age,gender,marital_status
count,14150.000000,13443,2046.0,12663.000000,14150.000000,297.0,14150,14000.000000,14150,14000.000000,14279.000000,14274.000000,14274,14274
unique,NaN,12,NaN,NaN,NaN,NaN,3,NaN,2,NaN,NaN,NaN,2,2
top,NaN,D00-SS,NaN,NaN,NaN,NaN,low,NaN,Employed,NaN,NaN,NaN,Male,Unmarried
freq,NaN,3905,NaN,NaN,NaN,NaN,6906,NaN,10784,NaN,NaN,NaN,9399,7298
mean,199.994346,NaN,1.0,0.718399,3.778304,1.0,NaN,0.621212,NaN,3.499357,112083.497864,32.892532,NaN,NaN
std,50.833697,NaN,0.0,0.173108,1.250162,0.0,NaN,0.250482,NaN,1.462584,8748.714222,9.974750,NaN,NaN
min,49.000000,NaN,1.0,0.316175,1.000000,1.0,NaN,0.040058,NaN,2.000000,0.000000,22.000000,NaN,NaN
25%,155.000000,NaN,1.0,0.563711,3.000000,1.0,NaN,0.450356,NaN,3.000000,105774.500000,24.000000,NaN,NaN
50%,199.000000,NaN,1.0,0.724731,4.000000,1.0,NaN,0.652394,NaN,3.000000,111296.000000,29.000000,NaN,NaN
75%,245.000000,NaN,1.0,0.871409,5.000000,1.0,NaN,0.824925,NaN,4.000000,116653.500000,41.000000,NaN,NaN


Need to merge the __df_departmentData__ to the __df_emp__ , first we have to correct the IT data from df_emp.

In [26]:
df_emp['department'].unique()

array([None, 'D00-SS', 'D00-SP', 'D00-MT', 'D00-PD', 'D00-IT', 'D00-AD',
       'D00-MN', 'D00-ENG', 'D00-PR', 'D00-TP', 'D00-FN', '-IT', nan],
      dtype=object)

In [36]:
df_emp['department'] = df_emp['department'].replace("-IT", "D00-IT")

In [37]:
df_emp['department'].unique()

array([None, 'D00-SS', 'D00-SP', 'D00-MT', 'D00-PD', 'D00-IT', 'D00-AD',
       'D00-MN', 'D00-ENG', 'D00-PR', 'D00-TP', 'D00-FN', nan],
      dtype=object)

Merge the Department column with respect to 

In [40]:
df = df_emp.merge(df_departmentData, how='outer', left_on='department', right_on='dept_id')
df.head(10)

,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,age,gender,marital_status,dept_id,dept_name,dept_head
0,246.0,None,NaN,0.866838,6.0,NaN,medium,0.134415,Left,4.0,124467,42.0,Female,Married,NaN,NaN,NaN
1,134.0,None,NaN,0.555718,2.0,NaN,low,0.511041,Left,3.0,112210,23.0,Female,Unmarried,NaN,NaN,NaN
2,155.0,None,NaN,0.464598,2.0,NaN,medium,0.414451,Left,3.0,127792,23.0,Female,Unmarried,NaN,NaN,NaN
3,156.0,None,NaN,NaN,2.0,NaN,medium,0.465747,Left,3.0,128509,25.0,Female,Unmarried,NaN,NaN,NaN
4,167.0,None,NaN,0.880053,5.0,NaN,low,0.682140,Employed,3.0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,233.0,None,NaN,0.965382,5.0,NaN,medium,0.962314,Left,5.0,125077,22.0,Female,Unmarried,NaN,NaN,NaN
6,260.0,None,NaN,1.000000,5.0,NaN,low,0.808455,Left,5.0,128184,23.0,Female,Unmarried,NaN,NaN,NaN
7,153.0,None,NaN,NaN,2.0,NaN,low,0.455101,Left,3.0,121337,25.0,Female,Unmarried,NaN,NaN,NaN
8,294.0,None,NaN,0.587699,3.0,NaN,low,0.760390,Left,4.0,119768,22.0,Female,Unmarried,NaN,NaN,NaN
9,246.0,None,NaN,0.884090,6.0,NaN,medium,0.131505,Left,4.0,115133,34.0,Female,Married,NaN,NaN,NaN


In [44]:
df.columns

Index(['avg_monthly_hrs', 'department', 'filed_complaint', 'last_evaluation',
       'n_projects', 'recently_promoted', 'salary', 'satisfaction', 'status',
       'tenure', 'employee_id', 'age', 'gender', 'marital_status', 'dept_id',
       'dept_name', 'dept_head'],
      dtype='object')

In [43]:
df.shape

(14279, 17)

We have to drop the duplicate column __department__ from the data frame.

In [47]:
df['department'].shape

(14279,)

In [51]:
df = df.drop('department', 1)

In [52]:
df.head()

,avg_monthly_hrs,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,age,gender,marital_status,dept_id,dept_name,dept_head
0,246.0,NaN,0.866838,6.0,NaN,medium,0.134415,Left,4.0,124467,42.0,Female,Married,NaN,NaN,NaN
1,134.0,NaN,0.555718,2.0,NaN,low,0.511041,Left,3.0,112210,23.0,Female,Unmarried,NaN,NaN,NaN
2,155.0,NaN,0.464598,2.0,NaN,medium,0.414451,Left,3.0,127792,23.0,Female,Unmarried,NaN,NaN,NaN
3,156.0,NaN,NaN,2.0,NaN,medium,0.465747,Left,3.0,128509,25.0,Female,Unmarried,NaN,NaN,NaN
4,167.0,NaN,0.880053,5.0,NaN,low,0.682140,Employed,3.0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df.shape

(14279, 16)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14279 entries, 0 to 14278
Data columns (total 16 columns):
avg_monthly_hrs      14150 non-null float64
filed_complaint      2046 non-null float64
last_evaluation      12663 non-null float64
n_projects           14150 non-null float64
recently_promoted    297 non-null float64
salary               14150 non-null object
satisfaction         14000 non-null float64
status               14150 non-null object
tenure               14000 non-null float64
employee_id          14279 non-null int64
age                  14274 non-null float64
gender               14274 non-null object
marital_status       14274 non-null object
dept_id              13443 non-null object
dept_name            13443 non-null object
dept_head            13443 non-null object
dtypes: float64(8), int64(1), object(7)
memory usage: 2.5+ MB


__Check missing data__

In [56]:
#missing data
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

                   Total   Percent
recently_promoted  13982  0.979200
filed_complaint    12233  0.856713
last_evaluation     1616  0.113173
dept_head            836  0.058548
dept_name            836  0.058548
dept_id              836  0.058548
tenure               279  0.019539
satisfaction         279  0.019539
status               129  0.009034
salary               129  0.009034
n_projects           129  0.009034
avg_monthly_hrs      129  0.009034
marital_status         5  0.000350
gender                 5  0.000350
age                    5  0.000350
employee_id            0  0.000000


We are generating Preprofiling report to get a brief overview of our data frame

In [60]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(df)
profile.to_file(outputfile="attritionData.html")

In [72]:
df['avg_monthly_hrs'].mean()

199.99434628975266

In [68]:
df['age'].median()

29.0

In [71]:
df['satisfaction'].median()

0.652394